In [1]:
import pandas as pd # DataFrame library
import os # Operating system library

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./secrets/my_credentials.json" # Credentials to be able to access the Google BigQuery public datasets

In [3]:
# This query extracts weather data from the US in Illinois from the year 1990 to 2023
with open("weather_query.sql", "r") as query_file:
    raw_weather_query = query_file.read()

# Load the data from the SQL query through Google BigQuery
raw_weather_data = pd.read_gbq(raw_weather_query, use_bqstorage_api = True, progress_bar_type = "tqdm_notebook", dialect = "standard")

/var/folders/j5/byfswq3j1bg3s3yk6x9qdzbr0000gp/T/ipykernel_14429/1168019163.py:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  raw_weather_data = pd.read_gbq(raw_weather_query, use_bqstorage_api = True, progress_bar_type = "tqdm_notebook", dialect = "standard")
/Users/henry/miniconda3/envs/weather/lib/python3.13/site-packages/pandas/io/gbq.py:207: LargeResultsWarning: Recommendation: Your results are 1.4 GiB. Consider using BigQuery DataFrames (https://bit.ly/bigframes-intro)to process large results with pandas compatible APIs with transparent SQL pushdown to BigQuery engine. This provides an opportunity to save on costs and improve performance. Please reach out to bigframes-feedback@google.com with any questions or concerns. To disable this message, run warnings.simplefilter('ignore', category=pandas_gbq.exceptions.LargeResultsWarning)
  

In [4]:
raw_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19102488 entries, 0 to 19102487
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         object 
 1   date       dbdate 
 2   time       object 
 3   element    object 
 4   value      float64
 5   qflag      object 
 6   latitude   float64
 7   longitude  float64
 8   name       object 
 9   elevation  float64
dtypes: dbdate(1), float64(4), object(5)
memory usage: 1.4+ GB


In [5]:
raw_weather_data.shape

(19102488, 10)

In [6]:
desired_types = {"id": "string", 
                 "date": "string", 
                 "time": "string", 
                 "name": "string", 
                 "element": "string", 
                 "value": "float", 
                 "latitude": "float", 
                 "longitude": "float",
                 "elevation": "float",
                 "qflag": "string"}

raw_weather_data = raw_weather_data.astype(desired_types)
raw_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19102488 entries, 0 to 19102487
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         string 
 1   date       string 
 2   time       string 
 3   element    string 
 4   value      float64
 5   qflag      string 
 6   latitude   float64
 7   longitude  float64
 8   name       string 
 9   elevation  float64
dtypes: float64(4), string(6)
memory usage: 1.4 GB


In [7]:
# Filter the data to only include the years 1990 to 2023
raw_weather_data = raw_weather_data[raw_weather_data["date"].str[:4].astype(int).between(1990, 2023)] 

In [8]:
raw_weather_data.to_parquet("./data/raw_weather.parquet", index = False)